# **Modelos Gráficos Probabilísticos - Trabalho Prático**

**Universidade Federal de Minas Gerais | Programa de Pós-Graduação em Ciência da Computação**

**Nome:** Leandro Augusto Lacerda Campos

Dezembro de 2019


## **1. Introdução**

O presente trabalho prático (TP) consiste em implementar, utilizando a linguagem de programação Python 3 e bibliotecas do *framework* Apache MXNet, a abordagem bayesiana para redes neurais recorrentes (RNNs) sugerida por Fortunato et al. [2017]. Nessa abordagem, os autores adicionam incerteza e regularização a RNNs aplicando o método *bayes by backprop* (BBB) formulado por Blundell et al. [2015].

Implementararemos, especificamente, a aplicação da abordagem proposta no problema de modelagem de linguagem. Nessa aplicação, Fortunato et al. [2017] utilizam o conjunto de dados de Marcus et al. [1993], denominado *Penn
Treebank* (PTB), e a arquitetura de rede indicada por Zaremba et al. [2014] para desenvolver uma RNN bayesiana que tem por objetivo predizer a próxima palavra de uma sequência.

Por fim, cabe ressaltar que não fazem parte do escopo desse TP:
*   a implementação do método de adaptação local da aproximação variacional a lotes de dados, denominado *posterior sharpening* e proposto por Fortunato et al. [2017];
*   a implementação do método de inferência dinâmica proposto por Mikolov et al. [2010]; e
*   a aplicação da abordagem proposta por Fortunato et al. [2017] para a tarefa de geração de legenda para
imagens.

### **1.1. Suposições sobre o leitor**

Assumimos que você, leitor, conhece os tópicos listados abaixo. Caso precise relembrá-los, incluímos algumas referências úteis:
*   Redes neurais: preparação dos dados, modelagem e treinamento. Consulte os capítulos [2](http://d2l.ai/chapter_preliminaries/index.html), [3](http://d2l.ai/chapter_linear-networks/index.html) e [4](http://d2l.ai/chapter_multilayer-perceptrons/index.html) do livro de Zhang et al. [2019].
*   Redes neurais recorrentes: preparação dos dados, arquitetura *long short-term memory* (LSTM), *gradient clipping* e a versão truncada do *backpropgation through time* (BPTT). Consulte os capítulos [8](http://d2l.ai/chapter_recurrent-neural-networks/index.html) e [9](http://d2l.ai/chapter_modern_recurrent-networks/index.html) do livro de Zhang et al. [2019].
*   Processamento de linguagem natural: *word embedding*. Consulte as quatro primeiras seções do capítulo [14](http://d2l.ai/chapter_natural-language-processing/index.html) do livro de Zhang et al. [2019].
*   Teoria da informação: entropia cruzada, perplexidade e divergência de Kullback-Leibler. Consulte as seções [17.11](http://d2l.ai/chapter_appendix_math/information-theory.html) e [8.4.4](http://d2l.ai/chapter_recurrent-neural-networks/rnn.html#perplexity) do livro de Zhang et al. [2019].
*   Inferência bayesiana. Consulte a seguinte [página](https://en.wikipedia.org/wiki/Bayesian_inference) do Wikipedia.

Também supomos que você tem experiência com o Apache MXNet (ou o PyTorch, que é similar) e Python 3.

### **1.2. Suposições sobre o ambiente de execução**

Esse TP foi planejado para ser executado em um ambiente do [Google Colab](https://colab.research.google.com/) com suporte a Python 3 e a unidade de processamento gráfico (GPU). Para verificar ou alterar as configurações desse notebook no Colab, clique no menu 'Edit' e depois em 'Notebook settings'. Vamos assumir que essa GPU é compatível com a versão 10.1 da NVIDIA CUDA e que esse *toolkit* já está instalado no ambiente.

O tempo de execução desse TP nesse ambiente é de aproximadamente 02h30min. Esse tempo pode variar dependendo das configurações do ambiente no qual você está conectado. Nas melhores configurações já testadas, o treinamento de cada época do modelo de Zaremba et al. [2014] demora cerca de 35 segundos. Caso você experimente um tempo 2x ou 3x maior do que esse, sugerimos que você se desconecte do atual ambiente de execução (clique no menu 'Runtime' e depois em 'Manage sessions') e então se conecte novamente.

Durante a execução desse notebook, você pode se deparar com os seguintes problemas:

*   Ocorreu um erro de processamento ou de memória na GPU. Nesse caso, recomendamos que você se desconecte do atual ambiente de execução (clique no menu 'Runtime' e depois em 'Manage sessions') e então se conecte novamente.
*   A conexão com o ambiente de execução foi interrompida por inatividade ou limite de tempo de uso excedido. Nessa situação, tente se conectar novamente clicando no botão 'Connect', situado no canto superior direito da página. As vezes, a execução recomeça de onde parou. Caso contrário, comece tudo novamente.

É importante ressaltar que você não precisa executar esse notebook para avaliar os resultados do TP, uma vez que a saída de cada célula está salva na versão publicada.

#### **1.2.1. Problemas com a GPU do Google Colab**

Com uma certa frequência, a GPU do ambiente de execução do Colab ao qual você se conecta não consegue executar corretamente esse notebook. Note que existem várias configurações de ambiente com suporte a GPU. Para fazer os últimos testes, foi necessário o uso de uma instância paga da plataforma Google Cloud. Essa instância tinha as seguintes configurações:

*   Tipo de máquina: n1-standard-8 (8 vCPUs, 30 GB de memória)
*   GPUs: 1 x NVIDIA Tesla V100
*   Zona: us-west1-b
*   Imagem: common-cu101-20191005
*   Disco de inicialização: Disco permanente SSD

Para conectar o Google Colab a uma instância paga do Google Cloud, siga os passos descritos nesse [vídeo](https://www.youtube.com/watch?v=U5HyNzf_ips).

Uma outra forma de tentar lidar com o problema da GPU no Colab é a seguinte:
*   Passo 1: Para treinar o modelo de Zaremba et al. [2014], transforme em comentário (coloque # no início de cada linha) o conteúdo da célula que instancia e treina o modelo de Fortunato et al. [2017]. E então execute o notebook.
*   Passo 2: Desfaça a ação do passo 1 e se desconecte do atual ambiente de execução (clique no menu 'Runtime' e depois em 'Manage sessions').
*   Passo 3: Para treinar o modelo de Fortunato et al. [2017], transforme em comentário (coloque # no início de cada linha) o conteúdo da célula que instancia e treina o modelo de Zaremba et al. [2014]. E então execute o notebook.

### **1.3. Bibliotecas e configurações globais**

Para executar esse TP, precisamos instalar e importar as seguintes bibliotecas.

In [0]:
!pip install mxnet-cu101==1.6.0b20190915

In [0]:
import collections
import math
import os
import random
import time
import urllib.request
import zipfile
import re

from mxnet import np, npx, nd, context, autograd, gluon, init, util
from mxnet.gluon import nn, rnn

npx.set_np()

A classe `Args` declara as configurações globais que nós utilizaremos para implementar e então treinar o modelo de tamanho médio de Zaremba et al. [2014] e o modelo de Fortunato et al. [2017].

In [0]:
class Args(object):
  # Data settings
  archive_file_name = 'ptb.zip'
  archive_file_root = './data'
  download_root = 'https://github.com/d2l-ai/d2l-en/raw/master/data'
  
  # Model settings
  embedding_size = 650
  hidden_size = 650
  num_layers = 2
  tie_weights = False
  save = './model.params'

  # Training settings
  init_scale = 0.05
  num_steps = 35
  batch_size = 20
  random_shift = False
  keep_prob = 0.5
  lr_start = 1.0
  lr_decay = 0.8
  clip_norm = 5
  num_epochs = 39
  high_lr_epochs = 6

  # BBB settings
  prior_pi = 0.25
  prior_sigma1 = np.exp(-1.0)
  prior_sigma2 = np.exp(-7.0)
  bbb_on_bias = False

  # Inference settings
  sample_mode = False
  
args = Args()

## **2. Dados de treinamento, validação e teste**

Nessa seção do TP, vamos mostrar como obter, pré-processar e carregar em mini-lotes os dados que compõem o *Penn Treebank* (PTB). Esse corpus linquístico é formado por artigos publicados no *Wall Street Journal* (WSJ) e está dividido em subconjuntos de treinamento, validação e teste. A função `read_ptb` é responsável por obter cada um desses subconjuntos.

In [0]:
def read_ptb(subset='train', root=args.archive_file_root):
  assert subset in ('train', 'valid', 'test'), \
    "Invalid subset %s; must be one of ['train', 'valid', 'test']"%subset

  archive_file_path = os.path.join(root, args.archive_file_name)
  
  if not os.path.isfile(archive_file_path):
    
    if not os.path.isdir(root):
      os.makedirs(root)
  
    archive_file_url = os.path.join(args.download_root,
                                    args.archive_file_name)
    urllib.request.urlretrieve(archive_file_url, archive_file_path)
    
  with zipfile.ZipFile(archive_file_path, mode='r') as f:
    data_file_name = 'ptb/ptb.{}.txt'.format(subset)
    raw_text = f.read(data_file_name).decode('utf-8').replace('\n', '<eos>')
  
  return raw_text.split()

In [0]:
train_subset, valid_subset, test_subset = [
  read_ptb(subset=subset)
  for subset in ['train', 'valid', 'test']
]

'# tokens in train_subset: {}'.format(len(train_subset))

'# tokens in train_subset: 929589'

Cada subconjunto é, inicialmente, uma sequência de sequências de palavras. Mas a função `read_ptb` o transforma em uma sequência de palavras. Veja, pela saída da célula acima, que o subconjunto de treinamento é representado, dessa forma, por uma sequência de 929.589 palavras.

A classe `Vocab`, definida abaixo, representa o conjunto das palavras que compõem um corpus linguístico e fornece duas bijeções. A primeira delas associa, a cada palavra desse conjunto, um número inteiro não-negativo, chamado índice. E a outra é simplesmente a inversa da anterior.

In [0]:
class Vocab(object):
  
  def __init__(self, subset):
    counter = collections.Counter(subset)
    self.token_freqs = sorted(counter.items(), key=lambda x: x[1], 
                              reverse=True)
    self.unk, uniq_tokens = 0, ['<unk>']
    uniq_tokens += [token for token, freq in self.token_freqs
                    if token not in uniq_tokens]
    self.idx_to_token, self.token_to_idx = [], dict()

    for token in uniq_tokens:
      self.idx_to_token.append(token)
      self.token_to_idx[token] = len(self.idx_to_token) - 1
    
  def __len__(self):
    return len(self.idx_to_token)
    
  def __getitem__(self, tokens):
    if not isinstance(tokens, (list, tuple)):
      return self.token_to_idx.get(tokens, self.unk)

    return [self.__getitem__(token) for token in tokens]
    
  def to_tokens(self, indices):
    if not isinstance(indices, (list, tuple)):
      return self.idx_to_token[indices]

    return [self.idx_to_token[index] for index in indices]

Vamos ver quantas palavras existem no vocabulário (ou dicionário) do subconjunto de treinamento.

In [0]:
vocab = Vocab(train_subset)
'vocab size: {}'.format(len(vocab))

'vocab size: 10000'

Agora vamos definir como cada subconjunto do corpus, representado por uma sequência de palavras, deve ser dividido em mini-lotes. Na função abaixo, chamada `bptt_batchify`, o parâmetro `batch_size` indica o número de subsequências da sequência `corpus` em cada mini-lote e o parâmetro `num_steps` assinala a quantidade de palavras por subsequência.

Seja $(X, Y)$ um dos mini-lotes retornados por `bptt_batchify`. O valor $X_{ij}$ representa a j-ésima palavra da i-ésima subsequência desse mini-lote. E o valor $Y_{ij}$ indica a palavra subsequente àquela representada por $X_{ij}$ na sequência `corpus`.

In [0]:
def bptt_batchify(corpus, num_steps, batch_size):
  num_indices = ((len(corpus) - 1) // batch_size) * batch_size
  Xs = np.array(corpus[:num_indices])
  Ys = np.array(corpus[1:(num_indices + 1)])
  Xs, Ys = Xs.reshape((batch_size, -1)), Ys.reshape((batch_size, -1))
  num_batches = Xs.shape[1] // num_steps

  for i in range(0, num_batches * num_steps, num_steps):
    X = Xs[:, i:(i+num_steps)]
    Y = Ys[:, i:(i+num_steps)]
    # X.shape = Y.shape = (batch_size, num_steps) 
    yield X, Y

Verifique, abaixo, que dois mini-lotes adjacentes retornados pela função `bptt_batchify` são também adjacentes na sequência fornecida como entrada. É por isso que o método implementado nessa função é chamado particionamento sequencial (apesar dele não induzir uma partição conforme definição usada em matemática).

In [0]:
my_seq = list(range(30))

for X, Y in bptt_batchify(my_seq, num_steps=6, batch_size=2):
    print('X:', X, '\nY:', Y)

X: [[ 0.  1.  2.  3.  4.  5.]
 [14. 15. 16. 17. 18. 19.]] 
Y: [[ 1.  2.  3.  4.  5.  6.]
 [15. 16. 17. 18. 19. 20.]]
X: [[ 6.  7.  8.  9. 10. 11.]
 [20. 21. 22. 23. 24. 25.]] 
Y: [[ 7.  8.  9. 10. 11. 12.]
 [21. 22. 23. 24. 25. 26.]]


A classe `SeqDataLoader`, definida abaixo, reúne os métodos de pré-processamento e de iteração, via mini-lotes, sobre os dados de um subconjunto do corpus.

In [0]:
class SeqDataLoader(object):
  
  def __init__(self, subset, vocab, num_steps=args.num_steps, 
               batch_size=args.batch_size, batchify_fn=bptt_batchify,
               random_shift=False):
    corpus = [vocab[token] for token in subset]
    shift = random.randint(0, num_steps) if random_shift else 0
    self.corpus = corpus[shift:]
    self.vocab = vocab
    self.num_steps = num_steps
    self.batch_size = batch_size
    self.num_batches = ((len(corpus) - 1) // batch_size) // num_steps
    self.get_iter = lambda: batchify_fn(self.corpus, num_steps, batch_size)

  def __iter__(self):
    return self.get_iter()

In [0]:
train_iter = SeqDataLoader(train_subset, vocab, random_shift=args.random_shift)
valid_iter = SeqDataLoader(valid_subset, vocab)
test_iter = SeqDataLoader(test_subset, vocab)

Pela saída da célula abaixo, podemos ver que cada mini-lote do subconjunto de treinamento contém 20 subsequências de 35 palavras cada.

In [0]:
for X, Y in train_iter:
  print('X.shape:', X.shape, '\nY.shape:', Y.shape)
  break

X.shape: (20, 35) 
Y.shape: (20, 35)


## **3. Modelo de Zaremba et al. [2014]**

O modelo proposto por Zaremba et al. [2014] consiste em uma camada *embedding* na entrada, um núcleo recorrente de arquitetura LSTM e uma camada densa na saída. Nesse TP, implementaremos apenas a configuração de tamanho médio desse modelo.

A camada *embedding* associa, a cada palavra do dicionário de um corpus linguístico, um vetor do espaço coordenado real de dimensão igual a `embedding_size`. Nesse modelo, a camada tem como entrada uma matriz de formato `(num_steps, batch_size)` e retorna um tensor de formato `(num_steps, batch_size, embedding_size)`.

In [0]:
encoder = nn.Embedding(input_dim=len(vocab),
                       output_dim=args.embedding_size)

In [0]:
encoder.initialize(init.Normal(sigma=args.init_scale), force_reinit=True)
encoded = encoder(X.T)
'encoded.shape: {}'.format(encoded.shape)

'encoded.shape: (35, 20, 650)'

O núcleo recorrente desse modelo é formado por `num_layers` camadas LSTM. O espaço-estado de cada uma delas tem dimensão igual a `hidden_size`. Esse núcleo, que modela a dependência sequencial entre as palavras, retorna dois tensores que representam, respectivamente, a saída, de formato `(num_steps, batch_size, hidden_size)`, e o par estado-memória, de formato `(2, num_layers, batch_size, hidden_size)`. Como entrada, ele recebe a saída da camada anterior e o valor inicial do par estado-memória.

In [0]:
lstm_layer = rnn.LSTM(hidden_size=args.hidden_size,
                      num_layers=args.num_layers,
                      dropout=1-args.keep_prob,
                      input_size=args.embedding_size)

In [0]:
lstm_layer.initialize(init.Normal(sigma=args.init_scale), force_reinit=True)
state = lstm_layer.begin_state(batch_size=args.batch_size)
'# states: {}, state[_].shape: {}'.format(len(state), state[0].shape)

'# states: 2, state[_].shape: (2, 20, 650)'

In [0]:
output, state = lstm_layer(encoded, state)
'output.shape: {}'.format(output.shape)

'output.shape: (35, 20, 650)'

Na saída desse modelo, temos uma camada densa. Ela recebe a saída do núcleo recorrente, após ter o seu formato alterado para `(num_steps * batch_size, hidden_size)`, e então retorna uma matriz de formato `(num_steps * batch_size, vocab_size)`. A aplicação linha por linha da função `softmax` nessa matriz nos dá `num_steps * batch_size` distribuições de probabilidade sobre as palavras do dicionário.

A classe `PTBModel` escapsula esse modelo, implementa a sua operação de *forward* e também define a função que retorna o valor inicial do par estado-memória. Como a operação de *backward* é inferida automaticamente pela biblioteca `autograd` do Apache MXNet, nós não precisamos nos preocupar com ela.

In [0]:
class PTBModel(nn.HybridBlock):

  def __init__(self, vocab_size, embedding_size=args.embedding_size,
               hidden_size=args.hidden_size, num_layers=args.num_layers,
               dropout=1-args.keep_prob, tie_weights=args.tie_weights, 
               **kwargs):
    super(PTBModel, self).__init__(**kwargs)

    with self.name_scope():
      self.drop = nn.Dropout(dropout)
      self.encoder = nn.Embedding(input_dim=vocab_size,
                                  output_dim=embedding_size)
      self.lstm = rnn.LSTM(hidden_size=hidden_size,
                           num_layers=num_layers,
                           dropout=dropout,
                           input_size=embedding_size)
      
      if tie_weights:
        self.decoder = nn.Dense(vocab_size, in_units=hidden_size,
                                params=self.encoder.params)
      else:
        self.decoder = nn.Dense(vocab_size, in_units=hidden_size)
            
      self.hidden_size = hidden_size

  def hybrid_forward(self, F, inputs, state):
    # inputs.shape = (batch_size, num_steps)
    # encoded.shape = (num_steps, batch_size, embedding_size)
    encoded = self.drop(self.encoder(inputs.T))
    # output.shape = (num_steps, batch_size, hidden_size)
    # state[_].shape = (num_layers, batch_size, hidden_size)
    output, state = self.lstm(encoded, state)
    output = self.drop(output)
    # decoded.shape = (num_steps * batch_size, vocab_size)
    decoded = self.decoder(output.reshape((-1, self.hidden_size)))
    return decoded, state
    
  def begin_state(self, *args, **kwargs):
    return self.lstm.begin_state(*args, **kwargs)

RNNs são propensas a problemas de extinção e explosão de gradientes. O primeiro deles pode ser remediado com o uso da arquitetura LSTM. E o segundo pode ser controlado com a utilização da função `grad_clipping`, definida abaixo, e do método BPTT truncado, que está implícito na forma como nós dividimos os subconjuntos do corpus em mini-lotes e no treinamento do modelo.

In [0]:
def grad_clipping(model, clip_norm):
  params = [p.data() for p in model.collect_params().values()]
  norm = math.sqrt(sum((p.grad ** 2).sum() for p in params))

  if norm > clip_norm:
    for param in params:
      param.grad[:] *= clip_norm / norm

O treinamento desse modelo está implementado nas funções `train`, `train_epoch` e `eval`, definidas nas três células a seguir. A principal estatística que elas retornam ou imprimem é a [perplexidade por palavra](https://en.wikipedia.org/wiki/Perplexity).

In [0]:
def eval(model, data_iter, loss, ctx):
  num_steps = data_iter.num_steps
  batch_size = data_iter.batch_size
  state = model.begin_state(batch_size=batch_size, ctx=ctx)
  loss_sum = 0
  steps_sum = 0

  for X, Y in data_iter:
    X = X.as_in_context(ctx)
    # Y.shape = (batch_size, num_steps)
    y = Y.T.reshape((-1,))
    # y.shape = (num_steps * batch_size)
    y = y.as_in_context(ctx)
    yhat, state = model(X, state)
    L = loss(yhat, y)
    # Sum over the sequence
    sequence_neg_log_prob = L.reshape((batch_size, num_steps)).sum(axis=1)
    # Average over the batch
    data_loss = sequence_neg_log_prob.mean()
    loss_sum += data_loss
    steps_sum += num_steps

  return loss_sum / steps_sum

In [0]:
def train_epoch(model, train_iter, loss, clip_norm, trainer, ctx):
  start_time = time.time()
  num_steps = train_iter.num_steps
  batch_size = train_iter.batch_size
  state = model.begin_state(batch_size=batch_size, ctx=ctx)
  loss_sum = 0
  steps_sum = 0
  
  for X, Y in train_iter:

    for s in state: s.detach()
    
    X = X.as_in_context(ctx)
    # Y.shape = (batch_size, num_steps)
    y = Y.T.reshape((-1,))
    # y.shape = (num_steps * batch_size)
    y = y.as_in_context(ctx)

    with autograd.record():
      yhat, state = model(X, state)
      L = loss(yhat, y)
      # Sum over the sequence
      sequence_neg_log_prob = L.reshape((batch_size, num_steps)).sum(axis=1)
      # Average over the batch
      data_loss = sequence_neg_log_prob.mean()
    
    data_loss.backward()
    grad_clipping(model, clip_norm)
    trainer.step(batch_size=1)
    loss_sum += data_loss
    steps_sum += num_steps
        
  return loss_sum / steps_sum, time.time() - start_time

In [0]:
def train(model, train_iter, valid_iter, test_iter, init_scale=args.init_scale, 
          lr=args.lr_start, lr_decay=args.lr_decay, num_epochs=args.num_epochs, 
          high_lr_epochs=args.high_lr_epochs, clip_norm=args.clip_norm, 
          ctx=context.cpu()):
  loss = gluon.loss.SoftmaxCrossEntropyLoss()
  model.initialize(ctx=ctx, force_reinit=True, 
                   init=init.Normal(sigma=init_scale))
  trainer = gluon.Trainer(model.collect_params(), 'sgd', 
                          {'learning_rate': lr})
  model.hybridize()
    
  # Train and check the progress
  for epoch in range(num_epochs):

    if epoch >= high_lr_epochs:
      lr = lr * lr_decay
      trainer._init_optimizer('sgd', {'learning_rate': lr})

    train_loss, speed = train_epoch(model, train_iter, loss, clip_norm, 
                                    trainer, ctx)
    print('[Epoch %d] time cost %.2fs, train loss %.2f, train ppl %.2f'%(
          epoch, speed, train_loss, math.exp(train_loss)))
    valid_loss = eval(model, valid_iter, loss, ctx)
    print('valid loss %.2f, valid ppl %.2f'%(valid_loss, math.exp(valid_loss)))
    test_loss = eval(model, test_iter, loss, ctx)
    print('test loss %.2f, test ppl %.2f'%(test_loss, math.exp(test_loss)))
  
  model.hybridize(active=False)

In [0]:
def try_gpu(device_id=0):
  if context.num_gpus() >= device_id + 1:
    return context.gpu(device_id)
  else:
    return context.cpu()

ctx = try_gpu()
print(ctx)

gpu(0)


A seguir, vamos instanciar o modelo e executar o seu treinamento. No artigo de Zaremba et al. [2014], a configuração de tamanho médio desse modelo obteve perplexidades por palavra de 86.2 e 82.7 nos subconjuntos de validação e teste, respectivamente. 

Uma vez que os pesos do modelo são inicializados de forma aleatória, os resultados podem variar cada vez que executamos o treinamento. O melhor resultado que obtivemos nesse notebook foi de 86.04 e 81.87 nos mesmos subconjuntos, nessa ordem.

In [0]:
model = PTBModel(len(vocab))
train(model, train_iter, valid_iter, test_iter, ctx=ctx)

[Epoch 0] time cost 22.38s, train loss 5.91, train ppl 367.70
valid loss 5.36, valid ppl 211.76
test loss 5.33, test ppl 207.05
[Epoch 1] time cost 22.08s, train loss 5.19, train ppl 179.45
valid loss 5.05, valid ppl 155.49
test loss 5.03, test ppl 152.29
[Epoch 2] time cost 21.87s, train loss 4.94, train ppl 140.39
valid loss 4.88, valid ppl 131.68
test loss 4.86, test ppl 128.88
[Epoch 3] time cost 21.99s, train loss 4.78, train ppl 119.35
valid loss 4.78, valid ppl 119.08
test loss 4.76, test ppl 117.02
[Epoch 4] time cost 21.86s, train loss 4.66, train ppl 105.65
valid loss 4.70, valid ppl 109.80
test loss 4.68, test ppl 107.53
[Epoch 5] time cost 21.97s, train loss 4.57, train ppl 96.10
valid loss 4.65, valid ppl 104.07
test loss 4.62, test ppl 101.82
[Epoch 6] time cost 21.95s, train loss 4.45, train ppl 85.39
valid loss 4.59, valid ppl 98.32
test loss 4.56, test ppl 95.85
[Epoch 7] time cost 21.88s, train loss 4.35, train ppl 77.31
valid loss 4.55, valid ppl 94.68
test loss 4.52

Um fato curioso é que o treinamento desse modelo pelos seus autores (que na época trabalhavam no Google Brain), no ano de 2014, consumiu meio dia de processamento, mesmo usando uma GPU. Hoje, utilizando um ambiente gratuito, executamos o mesmo treinamento em cerca de 23 minutos.

## **4. Abordagem bayesiana para redes neurais**

A abordagem bayesiana para redes neurais tem por finalidade representar a incerteza sistemática nessa classe de modelos. Ela consiste em atribuir ao vetor de pesos $\mathbf{w}$ da rede, que é uma constante numérica desconhecida, uma distribuição de probabilidade para expressar nossa expectativa, com base na informação disponível, de que dado valor para $\mathbf{w}$ é o verdadeiro. E ela nos ajuda a responder às seguintes perguntas: tendo em vista o tamanho e a diversidade do conjunto de treinamento, qual é a incerteza relacionada à estrutura e aos pesos de uma rede neural treinada nesse conjunto e qual é a confiança dessa rede ao fazer cada previsão?

Nessa abordagem, o treinamento de uma rede neural passa a ter como objetivo calcular a distribuição a posteriori de $\mathbf{w}$ dado o conjunto de treinamento $\mathcal{D}$, isto é, a distribuição $p(\mathbf{w} \mid \mathcal{D})$. E a distribuição preditiva de $\mathbf{y} \mid \mathbf{x}$ fica então definida por

$$\mathbb{E}_{p(\mathbf{w} \mid \mathcal{D})}[p(\mathbf{y} \mid \mathbf{x}, \mathbf{w})].$$

Como é inviável obter a posteriori $p(\mathbf{w} \mid \mathcal{D})$ em redes neurais de qualquer tamanho prático, Blundell et al. [2015] sugerem considerar uma família $\mathcal{Q} = {\{q(\mathbf{w} \mid \theta) : \theta \in \Theta\}}$ de distribuição paramétrica e então encontrar os parâmetros $\theta$ da distribuição $q(\mathbf{w} \mid \theta)$ que minimizam a sua divergência de Kullback-Leibler ($\text{KL}$) com a posteriori $p(\mathbf{w} \mid \mathcal{D})$:

$$\begin{aligned}
\theta^{*}
  & = \underset{\theta}{\operatorname{arg min}} \text{KL}(q(\mathbf{w} \mid \theta) \Vert p(\mathbf{w} \mid \mathcal{D})) \\
  & = \underset{\theta}{\operatorname{arg min}} \int q(\mathbf{w} \mid \theta) \log \frac{q(\mathbf{w} \mid \theta)} {p(\mathbf{w} \mid \mathcal{D})}d\mathbf{w} \\
  & = \underset{\theta}{\operatorname{arg min}} \int q(\mathbf{w} \mid \theta) \left[\log q(\mathbf{w} \mid \theta) - \frac{\log p(\mathcal{D} \mid \mathbf{w})p(\mathbf{w})}{p(\mathcal{D})}\right]d\mathbf{w} \\
  & = \underset{\theta}{\operatorname{arg min}} \int q(\mathbf{w} \mid \theta) \left[\log \frac{q(\mathbf{w} \mid \theta)}{p(\mathbf{w})} - \log p(\mathcal{D} \mid \mathbf{w}) + \log p(\mathcal{D})\right]d\mathbf{w} \\
  & = \underset{\theta}{\operatorname{arg min}} \text{KL}(q(\mathbf{w} \mid \theta) \Vert p(\mathbf{w})) - \mathbb{E}_{q(\mathbf{w} \mid \theta)}[\log p(\mathcal{D} \mid \mathbf{w})] + \log p(\mathcal{D}) \\
  & = \underset{\theta}{\operatorname{arg min}}\text{KL}(q(\mathbf{w} \mid \theta) \Vert p(\mathbf{w})) - \mathbb{E}_{q(\mathbf{w} \mid \theta)}[\log p(\mathcal{D} \mid \mathbf{w})], \\
\end{aligned}$$

onde o termo $\text{KL}(q(\mathbf{w} \mid \theta) \Vert p(\mathbf{w}))$ é chamado custo da complexidade e atua como um regularizador, e o termo $- \mathbb{E}_{q(\mathbf{w} \mid \theta)}[\log p(\mathcal{D} \mid \mathbf{w})]$ recebe o nome de custo da log-verossimilhança. Note que o primeiro termo não depende da saída da rede neural. Se assumirmos que os valores de $\mathbf{w}$ são coletivamente independentes, como é o caso, então podemos calcular o custo da complexidade camada por camada.

O modelo de Fortunato et al. [2017] fixa que a aproximação variacional $q(\mathbf{w} \mid \theta)$ é uma distribuição normal multivariada diagonal com parâmetros $\theta$ e que a distribuição a priori $p(\mathbf{w})$ é uma mistura de normais multivariadas diagonais, centradas na origem, com parâmetros `priori_pi`, `priori_sigma1` e `priori_sigma2`. As classes `CustomNormal` e `CustomScaleMixture` definem essas duas distribuições. 

Note que usamos `rho` ao invés de `sigma`, respectivamente $\rho$ e $\sigma$, para parametrizar uma distribuição normal multivariada. Nesse caso, definimos $\sigma = \log(1 + \exp(\rho))$, donde sempre vale $\sigma > 0$.

In [0]:
class CustomNormal(object):
  
  def __init__(self, F, mu, rho, shape=None):
    super(CustomNormal).__init__()
    self.mu = mu
    self.rho = rho
    
    if F is nd:
      self.normal = lambda : F.np.random.normal(size=rho.shape, ctx=rho.ctx)
    else:
      self.normal = lambda : F.np.random.normal(size=shape)
    
    self.log1p = F.np.log1p
    self.exp = F.np.exp
    self.log = F.np.log
  
  @property
  def sigma(self):
    return self.log1p(self.exp(self.rho))
  
  def sample(self):
    epsilon = self.normal()
    return self.mu + self.sigma * epsilon

  def _squared_difference(self, x, y):
    return (x - y) ** 2
  
  def log_prob(self, x):
    return (-0.5 * self.log(2. * math.pi) - self.log(self.sigma)
            -0.5 * self._squared_difference(x / self.sigma, 
                                            self.mu / self.sigma))

In [0]:
class CustomScaleMixture(object):

  def __init__(self, F, pi, sigma1, sigma2, ctx=None, dtype=None):
    super(CustomScaleMixture).__init__()

    if F is nd:
      to_array = lambda v : F.array(v, ctx=ctx, dtype=dtype).as_np_ndarray()
    else:
      to_array = lambda v : v
    
    self.log = F.np.log
    self.exp = F.np.exp
    self.max = F.np.max
    self.sum = F.np.sum
    self.squeeze = F.np.squeeze
    self.stack = F.np.stack
    
    self.mu, self.pi, self.sigma1, self.sigma2 = (
        to_array(v) for v in (0.0, pi, sigma1, sigma2))
    
    rho1 = self.log(self.exp(self.sigma1) - 1.0)
    rho2 = self.log(self.exp(self.sigma2) - 1.0)
    self.n1 = CustomNormal(F, self.mu, rho1)
    self.n2 = CustomNormal(F, self.mu, rho2)

  # This function is more numerically stable than log(sum(exp(x))).
  def _log_sum_exp(self, x, axis, keepdims=False):
    max_x = self.max(x, axis=axis, keepdims=True)
    x = self.log(self.sum(self.exp(x - max_x), axis=axis, keepdims=True))
    x = x + max_x

    if not keepdims:
      x = self.squeeze(x, axis=axis)
    
    return x
  
  def log_prob(self, x):
    mix1 = self.sum(self.n1.log_prob(x), -1) + self.log(self.pi)
    mix2 = self.sum(self.n2.log_prob(x), -1) + self.log(1.0 - self.pi)
    prior_mix = self.stack([mix1, mix2])
    lse_mix = self._log_sum_exp(prior_mix, [0])
    return self.sum(lse_mix)

Para inicializar os parâmetros relacionados a $\rho$ nas camadas bayesianas, precisamos definir a classe `Uniform` e as funções `non_lstm_rho_initializer` e `lstm_rho_initializer` conforme especificado por Fortunato et al. [2017].

In [0]:
class Uniform(init.Initializer):
  
  def __init__(self, low=-0.07, high=0.07):
    super(Uniform, self).__init__(low=low, high=high)
    self.low = low
    self.high = high

  def _init_weight(self, _, arr):
    np.random.uniform(self.low, self.high, arr.shape, out=arr)

In [0]:
def non_lstm_rho_initializer(prior_pi, prior_sigma1, prior_sigma2):
  prior_sigma = np.sqrt(prior_pi * (prior_sigma1 ** 2) + 
                        (1 - prior_pi) * (prior_sigma2 ** 2))
  minval = np.log(np.exp(prior_sigma / 2.0) - 1.0)
  maxval = np.log(np.exp(prior_sigma / 1.0) - 1.0)
  return Uniform(minval, maxval)

In [0]:
def lstm_rho_initializer(prior_pi, prior_sigma1, prior_sigma2):
  prior_sigma = np.sqrt(prior_pi * (prior_sigma1 ** 2) + 
                        (1 - prior_pi) * (prior_sigma2 ** 2))
  minval = np.log(np.exp(prior_sigma / 4.0) - 1.0)
  maxval = np.log(np.exp(prior_sigma / 2.0) - 1.0)
  return Uniform(minval, maxval)

### **4.1 Camadas bayesianas**

Para transformar o modelo de Zaremba et al. [2014] no modelo de Fortunato et al. [2017], precisamos definir uma versão bayesiana das camadas *embedding*, LSTM e densa. Para tanto, seguiremos os passos descritos nesse [artigo](https://mxnet.incubator.apache.org/api/python/docs/tutorials/extend/custom_layer.html) e utilizaremos, como ponto de partida, o código-fonte da versão original dessas camadas.

Nessa nova versão, os parâmetros de cada camada não são mais os seus respectivos pesos, mas sim os parâmetros da aproximação variacional da distribuição a posteriori desses pesos. Veja isso no construtor `__init__` de cada uma das três classes definidas a seguir. Porém, a principal diferença dessa versão em relação à original diz respeito à operação de *forward*. O novo procedimento pode ser resumido da seguinte forma:

*   **Passo 1**: Obtenha os pesos da camada. No modo treino ou no modo simulação, amostre os pesos da aproximação variacional. Caso contrário, tome o vetor de médias dessa distribuição.
*   **Passo 2**: Execute a operação de *forward* tal como na versão original da camada, usando os pesos obtidos no passo 1.
*   **Passo 3**: Calcule o custo da complexidade da camada, isto é, uma estimativa da divergência $\text{KL}$ da distribuição a priori dos pesos em relação à aproximação variacional, usando os pesos obtidos no passo 1.
*   **Passo 4**: Salve o custo da complexidade calculado no passo 3.
*   **Passo 5**: Retorne o resultado da operação executada no passo 2.

Essa operação de *forward* está implementada nos métodos `forward`, `hybrid_forward`[, `_forward_kernel`] e `_get_total_kl_cost` das classes definidas nessa seção.

Segue, na célula abaixo, a definição da classe `BayesianEmbedding`.


In [0]:
class BayesianEmbedding(nn.HybridBlock):

  def __init__(self, input_dim, output_dim, prior_pi, prior_sigma1, 
               prior_sigma2, dtype='float32', weight_mu_initializer=None, 
               weight_rho_initializer=None, sample_mode=False, **kwargs):
    super(BayesianEmbedding, self).__init__(**kwargs)
    self._input_dim = input_dim
    self._output_dim = output_dim
    self._prior_pi = prior_pi
    self._prior_sigma1 = prior_sigma1
    self._prior_sigma2 = prior_sigma2
    self._sample_mode = sample_mode
    self._total_kl_cost = None
    self._kwargs = {'input_dim': input_dim, 'output_dim': output_dim,
                    'dtype': dtype, 'sparse_grad': False}
    
    with self.name_scope():
      self.weight_mu = self.params.get('weight_mu',
                                       shape=(input_dim, output_dim),
                                       init=weight_mu_initializer,
                                       dtype=dtype, allow_deferred_init=True)
      self.weight_rho = self.params.get('weight_rho',
                                        shape=(input_dim, output_dim),
                                        init=weight_rho_initializer,
                                        dtype=dtype, allow_deferred_init=True)
  
  def __repr__(self):
    s = '{name}({input_dim} -> {output_dim}, {dtype})'
    return s.format(name=self.__class__.__name__, **self._kwargs)

  def forward(self, x, *args):
    emb, total_kl_cost = super(BayesianEmbedding, self).forward(x, *args)
    self._total_kl_cost = total_kl_cost
    return emb
  
  def hybrid_forward(self, F, x, weight_mu, weight_rho):
    weight_dist = CustomNormal(F, weight_mu, weight_rho, self.weight_rho.shape)

    if autograd.is_training() or self._sample_mode:
      weight = weight_dist.sample()
    else:
      weight = weight_dist.mu
    
    emb = F.npx.embedding(x, weight, name='fwd', **self._kwargs)
    # We could save computation here.
    total_kl_cost = self._get_total_kl_cost(F, weight_dist, weight)

    return emb, total_kl_cost
  
  def kl_cost(self, scale=1.0):
    assert self._total_kl_cost is not None, \
      'You must execute a forward operation before getting the KL cost'
    
    return self._total_kl_cost * scale
    
  def _get_total_kl_cost(self, F, weight_dist, weight):
    if F is nd:
      ctx = self.weight_mu.list_ctx()[0]
      dtype = self.weight_mu.dtype
      prior = CustomScaleMixture(F, self._prior_pi, self._prior_sigma1, 
                                self._prior_sigma2, ctx, dtype)
    else:
      prior = CustomScaleMixture(F, self._prior_pi, self._prior_sigma1, 
                                self._prior_sigma2)
    
    log_prior = prior.log_prob(weight)
    log_variational_posterior = weight_dist.log_prob(weight).sum()
    return log_variational_posterior - log_prior

  @property
  def sample_mode(self):
    return self._sample_mode
  
  @sample_mode.setter
  def sample_mode(self, value):
    self._sample_mode = value

Passemos, agora, para a definição da classe `BayesianLSTM`.

In [0]:
class BayesianLSTM(nn.HybridBlock):

  def __init__(self, input_size, hidden_size, prior_pi, prior_sigma1,
               prior_sigma2, num_layers=1, bidirectional=False, dtype='float32',
               i2h_weight_mu_initializer=None, i2h_weight_rho_initializer=None,
               h2h_weight_mu_initializer=None, h2h_weight_rho_initializer=None,
               i2h_bias_mu_initializer='zeros', i2h_bias_rho_initializer=None,
               h2h_bias_mu_initializer='zeros', h2h_bias_rho_initializer=None,
               sample_mode=False, **kwargs):
    super(BayesianLSTM, self).__init__(**kwargs)
    self._input_size = input_size
    self._hidden_size = hidden_size
    self._prior_pi = prior_pi
    self._prior_sigma1 = prior_sigma1
    self._prior_sigma2 = prior_sigma2
    self._num_layers = num_layers
    self._dir = 2 if bidirectional else 1
    self._dtype = dtype
    self._i2h_weight_mu_initializer = i2h_weight_mu_initializer
    self._i2h_weight_rho_initializer = i2h_weight_rho_initializer
    self._h2h_weight_mu_initializer = h2h_weight_mu_initializer
    self._h2h_weight_rho_initializer = h2h_weight_rho_initializer
    self._i2h_bias_mu_initializer = i2h_bias_mu_initializer
    self._i2h_bias_rho_initializer = i2h_bias_rho_initializer
    self._h2h_bias_mu_initializer = h2h_bias_mu_initializer
    self._h2h_bias_rho_initializer = h2h_bias_rho_initializer
    self._sample_mode = sample_mode
    self._params_shape = None
    self._total_kl_cost = None
    self._gates = 4 # number of gates in a LSTM layer
    self._layout = 'TNC' # T, N and C stand for sequence length, batch size, ... 
                         # and feature dimensions respectively.

    ng, ni, nh = self._gates, input_size, hidden_size
    for i in range(num_layers):
      for j in ['l', 'r'][:self._dir]:
        self._register_param('{}{}_i2h_weight_mu'.format(j, i),
                             shape=(ng*nh, ni),
                             init=i2h_weight_mu_initializer, dtype=dtype)
        self._register_param('{}{}_i2h_weight_rho'.format(j, i),
                             shape=(ng*nh, ni),
                             init=i2h_weight_rho_initializer, dtype=dtype)
        self._register_param('{}{}_h2h_weight_mu'.format(j, i),
                             shape=(ng*nh, nh),
                             init=h2h_weight_mu_initializer, dtype=dtype)
        self._register_param('{}{}_h2h_weight_rho'.format(j, i),
                             shape=(ng*nh, nh),
                             init=h2h_weight_rho_initializer, dtype=dtype)
        self._register_param('{}{}_i2h_bias_mu'.format(j, i),
                             shape=(ng*nh,),
                             init=i2h_bias_mu_initializer, dtype=dtype)
        self._register_param('{}{}_i2h_bias_rho'.format(j, i),
                             shape=(ng*nh,),
                             init=i2h_bias_rho_initializer, dtype=dtype)
        self._register_param('{}{}_h2h_bias_mu'.format(j, i),
                             shape=(ng*nh,),
                             init=h2h_bias_mu_initializer, dtype=dtype)
        self._register_param('{}{}_h2h_bias_rho'.format(j, i),
                             shape=(ng*nh,),
                             init=h2h_bias_rho_initializer, dtype=dtype)
      
      ni = nh * self._dir

  def _register_param(self, name, shape, init, dtype):
    p = self.params.get(name, shape=shape, init=init,
                        allow_deferred_init=True, dtype=dtype)
    setattr(self, name, p)
    return p

  def __repr__(self):
    s = '{name}({mapping}, {_layout}'

    if self._num_layers != 1:
      s += ', num_layers={_num_layers}'
    
    if self._dir == 2:
      s += ', bidirectional'
    
    s += ')'
    shape = self.l0_i2h_weight_mu.shape
    mapping = '{0} -> {1}'.format(shape[1] if shape[1] else None, 
                                  shape[0] // self._gates)
    return s.format(name=self.__class__.__name__, mapping=mapping, 
                    **self.__dict__)
    
  def _collect_params_with_prefix(self, prefix=''):
    if prefix:
      prefix += '.'

    pattern = re.compile(r'(l|r)(\d)_(i2h|h2h)_(weight|bias)_(mu|rho)\Z')
    
    def convert_key(m, bidirectional):
      d, l, g, t, p = [m.group(i) for i in range(1, 6)]

      if bidirectional:
        return '_unfused.{}.{}_cell.{}_{}_{}'.format(l, d, g, t, p)
      else:
        return '_unfused.{}.{}_{}_{}'.format(l, g, t, p)
    
    bidirectional = any(pattern.match(k).group(1) == 'r' 
                        for k in self._reg_params)
    ret = {prefix + convert_key(pattern.match(key), bidirectional) : val 
           for key, val in self._reg_params.items()}
    
    for name, child in self._children.items():
      ret.update(child._collect_params_with_prefix(prefix + name))
    
    return ret

  def state_info(self, batch_size=0):
    return [{'shape': (self._num_layers * self._dir, batch_size, 
                       self._hidden_size),
             '__layout__': 'LNC', 'dtype': self._dtype},
            {'shape': (self._num_layers * self._dir, batch_size, 
                       self._hidden_size),
             '__layout__': 'LNC', 'dtype': self._dtype}]

  def cast(self, dtype):
    super(BayesianLSTM, self).cast(dtype)
    self._dtype = dtype

  def begin_state(self, batch_size=0, func=nd.zeros, **kwargs):
    states = []

    for i, info in enumerate(self.state_info(batch_size)):

      if info is not None:
        info.update(kwargs)
      else:
        info = kwargs
      
      state = func(name='%sh0_%d' % (self.prefix, i), **info).as_np_ndarray()
      states.append(state)
    
    return states
  
  def __call__(self, inputs, states=None, **kwargs):
    self.skip_states = states is None
    if states is None:
      if isinstance(inputs, nd.NDArray):
        batch_size = inputs.shape[1] # TNC layout
        states = self.begin_state(batch_size, ctx=inputs.context, 
                                  dtype=inputs.dtype)
      else:
        states = self.begin_state(0, func=symbol.zeros)

    if isinstance(states, gluon.tensor_types):
      states = [states]

    return super(BayesianLSTM, self).__call__(inputs, states, **kwargs)

  def forward(self, x, *args):
    out = super(BayesianLSTM, self).forward(x, *args)
    # out = (outputs, states, total_kl_cost)
    self._total_kl_cost = out[2]
    return out[0] if self.skip_states else (out[0], out[1])
  
  def hybrid_forward(self, F, inputs, states, **kwargs):
    if F is nd:
      batch_size = inputs.shape[1] # TNC layout

    if F is nd:
      for state, info in zip(states, self.state_info(batch_size)):
        if state.shape != info['shape']:
          raise ValueError(
            "Invalid recurrent state shape. Expecting %s, got %s."%(
                                      str(info['shape']), str(state.shape)))

    return self._forward_kernel(F, inputs, states, **kwargs)

  def _forward_kernel(self, F, inputs, states, **kwargs):
    params_mu = (kwargs['{}{}_{}_{}'.format(d, l, g, t)].reshape(-1)
                 for t in ['weight_mu', 'bias_mu']
                 for l in range(self._num_layers)
                 for d in ['l', 'r'][:self._dir]
                 for g in ['i2h', 'h2h'])
    params_rho = (kwargs['{}{}_{}_{}'.format(d, l, g, t)].reshape(-1)
                  for t in ['weight_rho', 'bias_rho']
                  for l in range(self._num_layers)
                  for d in ['l', 'r'][:self._dir]
                  for g in ['i2h', 'h2h'])
    params_mu = F.np._internal.rnn_param_concat(*params_mu, dim=0)
    params_rho = F.np._internal.rnn_param_concat(*params_rho, dim=0)
    
    if self._params_shape is None and F is nd:
      self._params_shape = params_rho.shape

    params_dist = CustomNormal(F, params_mu, params_rho, self._params_shape)

    if autograd.is_training() or self._sample_mode:
      params = params_dist.sample()
    else:
      params = params_dist.mu
    
    rnn_args = states

    rnn = F.npx.rnn(inputs, params, *rnn_args, use_sequence_length=False,
                    state_size=self._hidden_size, projection_size=None,
                    num_layers=self._num_layers, bidirectional=self._dir == 2,
                    p=0, state_outputs=True, mode='lstm',
                    lstm_state_clip_min=None,
                    lstm_state_clip_max=None,
                    lstm_state_clip_nan=False)

    outputs, states = rnn[0], [rnn[1], rnn[2]]
    # We could save computation here.
    total_kl_cost = self._get_total_kl_cost(F, params_dist, params)

    return outputs, states, total_kl_cost

  def kl_cost(self, scale=1.0):
    assert self._total_kl_cost is not None, \
      'You must execute a forward operation before getting the KL cost'

    return self._total_kl_cost * scale
    
  def _get_total_kl_cost(self, F, params_dist, params):
    if F is nd:
      ctx = self.l0_i2h_weight_mu.list_ctx()[0]
      dtype = self.l0_i2h_weight_mu.dtype
      prior = CustomScaleMixture(F, self._prior_pi, self._prior_sigma1, 
                                self._prior_sigma2, ctx, dtype)
    else:
      prior = CustomScaleMixture(F, self._prior_pi, self._prior_sigma1, 
                                self._prior_sigma2)

    log_prior = prior.log_prob(params)
    log_variational_posterior = params_dist.log_prob(params).sum()
    return log_variational_posterior - log_prior
  
  @property
  def sample_mode(self):
    return self._sample_mode
  
  @sample_mode.setter
  def sample_mode(self, value):
    self._sample_mode = value

Por fim, vamos definir a classe `BayesianDense`.

In [0]:
class BayesianDense(nn.HybridBlock):

  def __init__(self, units, in_units, prior_pi, prior_sigma1, prior_sigma2,
               activation=None, use_bias=True, flatten=True, dtype='float32',
               weight_mu_initializer=None, weight_rho_initializer=None, 
               bias_mu_initializer='zeros', bias_rho_initializer=None,
               sample_mode=False, bbb_on_bias=True, **kwargs):
    super(BayesianDense, self).__init__(**kwargs)
    self._units = units
    self._in_units = in_units
    self._prior_pi = prior_pi
    self._prior_sigma1 = prior_sigma1
    self._prior_sigma2 = prior_sigma2
    self._flatten = flatten
    self._sample_mode = sample_mode
    self._total_kl_cost = None
    
    with self.name_scope():
      self.weight_mu = self.params.get('weight_mu',
                                       shape=(units, in_units),
                                       init=weight_mu_initializer,
                                       dtype=dtype, allow_deferred_init=True)
      self.weight_rho = self.params.get('weight_rho',
                                        shape=(units, in_units),
                                        init=weight_rho_initializer,
                                        dtype=dtype, allow_deferred_init=True)
      
      if use_bias:
        self.bias_mu = self.params.get('bias_mu', shape=(units,),
                                       init=bias_mu_initializer,
                                       dtype=dtype, allow_deferred_init=True)
        
        if bbb_on_bias:
          self.bias_rho = self.params.get('bias_rho', shape=(units,),
                                          init=bias_rho_initializer,
                                          dtype=dtype, allow_deferred_init=True)
        else:
          self.bias_rho = None

      else:
        self.bias_mu = None
        self.bias_rho = None
      
      if activation is not None:
        self.act = nn.Activation(activation, prefix=activation+'_')
      else:
        self.act = None
  
  def __repr__(self):
    s = '{name}({layout}, {act})'
    shape = self.weight_mu.shape
    return s.format(name=self.__class__.__name__,
                    act=self.act if self.act else 'linear',
                    layout='{0} -> {1}'.format(shape[1] if shape[1] else None, 
                                               shape[0]))

  def forward(self, x, *args):
    act, total_kl_cost = super(BayesianDense, self).forward(x, *args)
    self._total_kl_cost = total_kl_cost
    return act
  
  def hybrid_forward(self, F, x, weight_mu, weight_rho, 
                     bias_mu=None, bias_rho=None):
    weight_dist = CustomNormal(F, weight_mu, weight_rho, self.weight_rho.shape)

    if autograd.is_training() or self._sample_mode:
      weight = weight_dist.sample()
    else:
      weight = weight_dist.mu

    if bias_mu is not None:

      if bias_rho is not None:
        bias_dist = CustomNormal(F, bias_mu, bias_rho, self.bias_rho.shape)

        if autograd.is_training() or self._sample_mode:
          bias = bias_dist.sample()
        else:
          bias = bias_dist.mu

      else:
        bias = bias_mu

    else:
      bias = None

    act = F.npx.fully_connected(x, weight, bias, no_bias=bias_mu is None, 
                                num_hidden=self._units, flatten=self._flatten, 
                                name='fwd')

    if self.act is not None:
      act = self.act(act)
    
    # We could save computation here.
    if bias_rho is not None:
      total_kl_cost = self._get_total_kl_cost(F, weight_dist, weight, 
                                              bias_dist, bias)
    else:
      total_kl_cost = self._get_total_kl_cost(F, weight_dist, weight)

    return act, total_kl_cost
  
  def kl_cost(self, scale=1.0):
    assert self._total_kl_cost is not None, \
      'You must execute a forward operation before getting the KL cost'

    return self._total_kl_cost * scale

  def _get_total_kl_cost(self, F, weight_dist, weight, 
                         bias_dist=None, bias=None):
    if F is nd:
      ctx = self.weight_mu.list_ctx()[0]
      dtype = self.weight_mu.dtype
      prior = CustomScaleMixture(F, self._prior_pi, self._prior_sigma1, 
                                self._prior_sigma2, ctx, dtype)
    else:
      prior = CustomScaleMixture(F, self._prior_pi, self._prior_sigma1, 
                                self._prior_sigma2)

    if bias_dist is not None:
      log_prior = prior.log_prob(weight) + prior.log_prob(bias)
      log_variational_posterior = weight_dist.log_prob(weight).sum() + \
                                  bias_dist.log_prob(bias).sum()
    else:
      log_prior = prior.log_prob(weight)
      log_variational_posterior = weight_dist.log_prob(weight).sum()
    
    return log_variational_posterior - log_prior
  
  @property
  def sample_mode(self):
    return self._sample_mode
  
  @sample_mode.setter
  def sample_mode(self, value):
    self._sample_mode = value

## **5. Modelo de Fortunato et al. [2017]**

O modelo de Fortunato et al. [2017] se diferencia do anterior nos seguintes aspectos:

*   Não há dropout.
*   As camadas originais foram substituídas pelas suas respectivas versões bayesianas, definidas anteriormente.
*   Esse modelo tem um método chamado `kl_cost` que retorna a soma do custo da complexidade de cada camada que compõe esse modelo.

Seguindo o que foi proposto pelos autores, não atribuímos distribuição de probabilidade sobre o bias da camada densa, conforme valor corrente da variável `bbb_on_bias`.

In [0]:
class BayesianPTBModel(nn.HybridBlock):

  def __init__(self, vocab_size, embedding_size=args.embedding_size,
               hidden_size=args.hidden_size, num_layers=args.num_layers,
               prior_pi=args.prior_pi, prior_sigma1=args.prior_sigma1, 
               prior_sigma2=args.prior_sigma2, tie_weights=args.tie_weights,
               sample_mode=args.sample_mode, bbb_on_bias=args.bbb_on_bias,
               **kwargs):
    super(BayesianPTBModel, self).__init__(**kwargs)
    self._sample_mode = sample_mode
    self._total_kl_cost = None
    non_lstm_rho_init = non_lstm_rho_initializer(prior_pi, prior_sigma1, 
                                                 prior_sigma2)
    lstm_rho_init = lstm_rho_initializer(prior_pi, prior_sigma1, prior_sigma2)

    with self.name_scope():
      self.encoder = BayesianEmbedding(input_dim=vocab_size,
                                       output_dim=embedding_size, 
                                       prior_pi=prior_pi,
                                       prior_sigma1=prior_sigma1, 
                                       prior_sigma2=prior_sigma2,
                                       weight_rho_initializer=non_lstm_rho_init,
                                       sample_mode=sample_mode)
      self.lstm = BayesianLSTM(input_size=embedding_size,
                               hidden_size=hidden_size,
                               prior_pi=prior_pi,
                               prior_sigma1=prior_sigma1,
                               prior_sigma2=prior_sigma2,
                               num_layers=num_layers,
                               i2h_weight_rho_initializer=lstm_rho_init,
                               h2h_weight_rho_initializer=lstm_rho_init,
                               i2h_bias_rho_initializer=lstm_rho_init,
                               h2h_bias_rho_initializer=lstm_rho_init,
                               sample_mode=sample_mode)
      
      if tie_weights:
        self.decoder = BayesianDense(units=vocab_size,
                                     in_units=hidden_size,
                                     prior_pi=prior_pi,
                                     prior_sigma1=prior_sigma1,
                                     prior_sigma2=prior_sigma2,
                                     weight_rho_initializer=non_lstm_rho_init,
                                     bias_rho_initializer=non_lstm_rho_init,
                                     sample_mode=sample_mode,
                                     bbb_on_bias=bbb_on_bias,
                                     params=self.encoder.params)
      else:
        self.decoder = BayesianDense(units=vocab_size,
                                     in_units=hidden_size,
                                     prior_pi=prior_pi,
                                     prior_sigma1=prior_sigma1,
                                     prior_sigma2=prior_sigma2,
                                     weight_rho_initializer=non_lstm_rho_init,
                                     bias_rho_initializer=non_lstm_rho_init,
                                     sample_mode=sample_mode,
                                     bbb_on_bias=bbb_on_bias)

      self.hidden_size = hidden_size

  def forward(self, x, *args):
    out = super(BayesianPTBModel, self).forward(x, *args)
    # out = (outputs, states, total_kl_cost)
    self._total_kl_cost = out[2]
    return out[0], out[1]

  def hybrid_forward(self, F, inputs, state):
    # inputs.shape = (batch_size, num_steps)
    # encoded.shape = (num_steps, batch_size, embedding_size)
    encoded = self.encoder(inputs.T)
    # output.shape = (num_steps, batch_size, hidden_size)
    # state[_].shape = (num_layers, batch_size, hidden_size)
    output, state = self.lstm(encoded, state)
    # decoded.shape = (num_steps * batch_size, vocab_size)
    decoded = self.decoder(output.reshape((-1, self.hidden_size)))
    total_kl_cost = self.encoder.kl_cost() + self.lstm.kl_cost() + \
                    self.decoder.kl_cost()
    return decoded, state, total_kl_cost
    
  def begin_state(self, *args, **kwargs):
    return self.lstm.begin_state(*args, **kwargs)
  
  def kl_cost(self, scale=1.0):
    assert self._total_kl_cost is not None, \
      'You must execute a forward operation before getting the KL cost'

    return self._total_kl_cost * scale
  
  @property
  def sample_mode(self):
    return self._sample_mode
  
  @sample_mode.setter
  def sample_mode(self, value):
    self._sample_mode = value
    self.encoder.sample_mode = value
    self.lstm.sample_mode = value
    self.decoder.sample_mode = value

As configurações de treinamento desse modelo são ligeiramente diferentes das usadas no modelo anterior. Veja, abaixo, o que precisa ser alterado.

In [0]:
args.lr_decay = 0.9
args.num_epochs = 70
args.high_lr_epochs = 20

No treinamento desse modelo, vamos estimar a divergência $\text{KL}$ que queremos minimizar utilizando simulação Monte Carlo com apenas uma amostra gerada da aproximação variacional. E para validação e teste, os pesos utilizados na operação de *forward* são o vetor de médias dessa distribuição convergente. Fortunato et al. [2017] utilizaram esse mesmo procedimento para evitar maior custo computacional em comparação com o custo do modelo de Zaremba et al. [2014].

Como o custo da complexidade da rede não depende dos dados, nós precisamos colocá-lo na mesma escala do custo da log-verossimilhança em cada mini-lote. De acordo com Blundell et al. [2015], existem várias formas de se fazer esse ajuste. A forma usada por Fortunato et al. [2017] pode ser considerada a mais simples e direta: basta dividir `total_kl_cost` por `batch_size * num_batches`. Desse modo, o custo da complexidade é distribuído uniformimente entre todas as subsequências de todos os mini-lotes do conjunto de treinamento.

In [0]:
def train_epoch_bbb(model, train_iter, loss, clip_norm, trainer, ctx):
  start_time = time.time()
  num_steps = train_iter.num_steps
  batch_size = train_iter.batch_size
  num_batches = train_iter.num_batches
  num_dataset_elements = batch_size * num_batches
  state = model.begin_state(batch_size=batch_size, ctx=ctx)
  loss_sum = 0
  steps_sum = 0
  
  for X, Y in train_iter:

    for s in state: s.detach()
    
    X = X.as_in_context(ctx)
    # Y.shape = (batch_size, num_steps)
    y = Y.T.reshape((-1,))
    # y.shape = (num_steps * batch_size)
    y = y.as_in_context(ctx)

    with autograd.record():
      yhat, state = model(X, state)
      L = loss(yhat, y)
      # Sum over the sequence
      sequence_neg_log_prob = L.reshape((batch_size, num_steps)).sum(axis=1)
      # Average over the batch
      data_loss = sequence_neg_log_prob.mean()
      total_kl_cost = model.kl_cost()
      scaled_kl_cost = total_kl_cost / num_dataset_elements
      # KL divergence
      total_loss = scaled_kl_cost + data_loss
    
    total_loss.backward()
    grad_clipping(model, clip_norm)
    trainer.step(batch_size=1)
    loss_sum += data_loss
    steps_sum += num_steps
        
  return loss_sum / steps_sum, time.time() - start_time

In [0]:
def train_bbb(model, train_iter, valid_iter, test_iter, 
              init_scale=args.init_scale, lr=args.lr_start, 
              lr_decay=args.lr_decay, num_epochs=args.num_epochs, 
              high_lr_epochs=args.high_lr_epochs, clip_norm=args.clip_norm, 
              ctx=context.cpu()):
  loss = gluon.loss.SoftmaxCrossEntropyLoss()
  model.initialize(ctx=ctx, force_reinit=True,
                   init=init.Normal(sigma=init_scale))
  trainer = gluon.Trainer(model.collect_params(), 'sgd', 
                          {'learning_rate': lr})
    
  # Train and check the progress
  for epoch in range(num_epochs):

    if epoch >= high_lr_epochs:
      lr = lr * lr_decay
      trainer._init_optimizer('sgd', {'learning_rate': lr})

    train_loss, speed = train_epoch_bbb(model, train_iter, loss, clip_norm, 
                                        trainer, ctx)
    print('[Epoch %d] time cost %.2fs, train loss %.2f, train ppl %.2f'%(
          epoch, speed, train_loss, math.exp(train_loss)))
    valid_loss = eval(model, valid_iter, loss, ctx)
    print('valid loss %.2f, valid ppl %.2f'%(valid_loss, math.exp(valid_loss)))
    test_loss = eval(model, test_iter, loss, ctx)
    print('test loss %.2f, test ppl %.2f'%(test_loss, math.exp(test_loss)))

Para finalizar, vamos instanciar esse modelo e executar o seu treinamento. A perplexidade por palavra reportada no artigo de Fortunato et al. [2017] foi de 78.8 no subconjunto de validação e de 75.5 no de teste.

Uma vez que os pesos do modelo são inicializados de forma aleatória, os resultados podem variar cada vez que executamos o treinamento. O melhor resultado que obtivemos nesse notebook foi de 79.37 e 75.98 nos mesmos subconjuntos, nessa ordem. Executando mais vezes esse procedimento, provavelmente serão obtidos resultados melhores. Mas nós não fizemos mais tentativas por questão de tempo (e de custo).

In [0]:
model = BayesianPTBModel(len(vocab))
train_bbb(model, train_iter, valid_iter, test_iter, ctx=ctx)

[Epoch 0] time cost 62.14s, train loss 6.29, train ppl 538.67
valid loss 5.65, valid ppl 284.94
test loss 5.62, test ppl 277.05
[Epoch 1] time cost 61.87s, train loss 5.43, train ppl 227.63
valid loss 5.31, valid ppl 201.71
test loss 5.28, test ppl 196.38
[Epoch 2] time cost 62.25s, train loss 5.12, train ppl 168.16
valid loss 5.07, valid ppl 159.37
test loss 5.04, test ppl 155.23
[Epoch 3] time cost 62.61s, train loss 4.95, train ppl 140.64
valid loss 4.96, valid ppl 142.04
test loss 4.93, test ppl 138.00
[Epoch 4] time cost 62.45s, train loss 4.83, train ppl 124.66
valid loss 4.87, valid ppl 130.52
test loss 4.84, test ppl 127.08
[Epoch 5] time cost 62.24s, train loss 4.73, train ppl 113.61
valid loss 4.80, valid ppl 121.94
test loss 4.77, test ppl 118.45
[Epoch 6] time cost 62.75s, train loss 4.66, train ppl 105.86
valid loss 4.76, valid ppl 117.33
test loss 4.74, test ppl 114.60
[Epoch 7] time cost 62.68s, train loss 4.61, train ppl 100.15
valid loss 4.73, valid ppl 113.07
test los

## **6. Considerações finais**

Segue, abaixo, uma lista dos próximos passos desse trabalho. A existência dessa lista, entretanto, não implica que os objetivos desse TP não foram alcançados. Com efeito, conseguimos alcançar resultados similares aos de Zaremba et al. [2014] e Fortunato et al. [2017] nas configurações de interesse. Mas com o conhecimento que foi adquirido ao longo desse trabalho, agora é possível fazer mais e melhor.

*   Suportar simulação Monte Carlo no modo de inferência da rede neural bayesiana. Na presente versão desse TP, os pesos da rede que são usados durante a inferência são o vetor de médias da aproximação variacional.
*   Suportar `num_samples` amostras na simulação Monte Carlo dos modos de treinamento e inferência da rede neural bayesiana. Na atual versão desse TP, usamos apenas uma amostra por operação de *forward* durante o treinamento da rede.
*   Concluir suporte a programação simbólica na rede neural bayesiana. Desse modo, será possível compilar o grafo computacional da operação de *forward* dessa rede e melhorar a sua eficiência computacional, tanto em termos de tempo de execução quanto de uso de memória. Atualmente, se você chamar o método `hybridize` do modelo bayesiano, não será possível treinar corretamente a rede.
*   Tornar a interface e o código da rede neural bayesiana mais simples, organizado, confiável, manutenível e expansível. Com o conhecimento adquirido sobre a abordagem bayesiana para redes neurais e sobre o funcionamento e a organização do Apache MXNet, acredito ser possível desenvolver uma forma simples de combinar essa abordagem com muitos modelos de aprendizado profundo.
*   Explorar utilização da informação sobre incerteza que é retornada pela rede neural bayesiana. Por exemplo, poda de parâmetros.



## **7. Referências bibliográficas**

BLUNDELL, Charles et al. Weight uncertainty in neural networks. **arXiv preprint arXiv:1505.05424**, 2015.

FORTUNATO, Meire; BLUNDELL, Charles; VINYALS, Oriol. Bayesian recurrent neural networks. **arXiv preprint arXiv:1704.02798**, 2017.

MARCUS, Mitchell; SANTORINI, Beatrice; MARCINKIEWICZ, Mary Ann. Building a large annotated corpus of English: The Penn Treebank. 1993.

MIKOLOV, Tomáš et al. Recurrent neural network based language model. In: **Eleventh annual conference of the international speech communication association**. 2010.

ZAREMBA, Wojciech; SUTSKEVER, Ilya; VINYALS, Oriol. Recurrent neural network regularization. **arXiv preprint arXiv:1409.2329**, 2014.

ZHANG, Aston et al. Dive into Deep Learning. 2019.